# Valid Sudoku

Determine if a 9 x 9 Sudoku board is valid. Only the filled cells need to be validated according to the following rules:

Each row must contain the digits 1-9 without repetition.
Each column must contain the digits 1-9 without repetition.
Each of the nine 3 x 3 sub-boxes of the grid must contain the digits 1-9 without repetition.
Note:

A Sudoku board (partially filled) could be valid but is not necessarily solvable.
Only the filled cells need to be validated according to the mentioned rules.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/f/ff/Sudoku-by-L2G-20050714.svg/250px-Sudoku-by-L2G-20050714.svg.png)

```
Input: board = 
[["5","3",".",".","7",".",".",".","."]
,["6",".",".","1","9","5",".",".","."]
,[".","9","8",".",".",".",".","6","."]
,["8",".",".",".","6",".",".",".","3"]
,["4",".",".","8",".","3",".",".","1"]
,["7",".",".",".","2",".",".",".","6"]
,[".","6",".",".",".",".","2","8","."]
,[".",".",".","4","1","9",".",".","5"]
,[".",".",".",".","8",".",".","7","9"]]
Output: true
```

```
Input: board = 
[["8","3",".",".","7",".",".",".","."]
,["6",".",".","1","9","5",".",".","."]
,[".","9","8",".",".",".",".","6","."]
,["8",".",".",".","6",".",".",".","3"]
,["4",".",".","8",".","3",".",".","1"]
,["7",".",".",".","2",".",".",".","6"]
,[".","6",".",".",".",".","2","8","."]
,[".",".",".","4","1","9",".",".","5"]
,[".",".",".",".","8",".",".","7","9"]]
Output: false
Explanation: Same as Example 1, except with the 5 in the top left corner being modified to 8. Since there are two 8's in the top left 3x3 sub-box, it is invalid.
```

## Approaches
row check and column check and sub-grid check

row check: 
    each and every row only has numbers between 1-9 & no repeats

column check: 
    convert columns to rows
        column i = ith element of every row
    run row check

sub-grid check:
    convert sub-grid to rows 
        0 = (0, 0) (0, 1) (0, 2) (1, 0) (1, 1) (1, 2) (2, 0) (2, 1) (2, 2)
        1 = (0, 3) (0, 4) (0, 5) (1, 3) (1, 4) (1, 5) (2, 3) (2, 4) (2, 4)
        2 = (0, 6) (0, 7) (0, 8) (1, 6) (1, 7) (1, 8) (2, 6) (2, 7) (2, 8)

        3 = (3, 0) (3, 1) (3, 2) (4, 0) (4, 1) (4, 2) (5, 0) (5, 1) (5, 2)

        rows for i = floor (i / 3) * 3 ... ceil (i / 3) * 3
        cols for i = i % 3 * 3 ... (i % 3) + 1 * 3

        sub-grid i = rows ()

In [ ]:
from collections import defaultdict
from math import floor, ceil

def isValidSudoku(board):
    def rowCheck(row):
        row = [int(r) for r in row if r != "."]
        freq = defaultdict(int)
        for x in row:
            freq[x] = freq[x] + 1
        return all([key < 10 and val < 2 for (key, val) in freq.items()])

    def convertCols(board):
        return [[board[j][i] for j in range(9)] for i in range(9)]
    
    def convertSubGrid(board):
        def genRows(i): return range(floor(i / 3) * 3, ceil(i / 3) * 3) if i % 3 != 0 else range(floor(i/3) * 3, (floor(i/3)+1)*3) 
        def genCols(i): return range(i%3 *3, (i%3+1)*3)
        return [[board[i][j] for i in genRows(n) for j in genCols(n)] for n in range(9)]

    rows = all([rowCheck(r) for r in board])
    cols = all([rowCheck(r) for r in convertCols(board)])
    subGrid = all([rowCheck(r) for r in convertSubGrid(board)])

    return rows and cols and subGrid

## Approach 2

- a subGrid can be identified by 2 coordinates (p, k) and elements in (i, j) where p = floor(i/3) and k = floor(j/3)

- for all possible i, j:
    - checks:
        - is element i, j < 10
        - has row i seen element at i, j
        - has col j seen element at i, j
        - has subGrid (floor(i/3), floor(j/3)) seen element at i, j


In [36]:
from math import floor
from collections import defaultdict
from functools import reduce

def isValidSudoku1(board):
    allIJs = [(i, j) for i in range(9) for j in range(9)]
    def check(sets, coordinates):
        (i, j) = coordinates
        elem = int(board[i][j]) if board[i][j] != "." else None
        if sets == None: return None
        if elem == None: return sets
        (rowSets, colSets, subGridSets) = sets
        rowCheck = elem not in rowSets[i]
        colCheck = elem not in colSets[j]
        subGridCheck = elem not in subGridSets[(floor(i/3), floor(j/3))]
        if rowCheck and colCheck and subGridCheck:
            rowSets[i].add(elem)
            colSets[j].add(elem)
            subGridSets[(floor(i/3), floor(j/3))].add(elem)
            return sets
        return None
    return True if reduce(check, allIJs, (defaultdict(set), defaultdict(set), defaultdict(set))) != None else False

print(isValidSudoku1(board))

False
